<a href="https://colab.research.google.com/github/fzorrilla-ml/First-Order-Motion-Model-for-Image-Animation/blob/main/first_order_model_demo_rt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "First Order Motion Model for Image Animation" versión simplificada por RT

**Clonar el repositorio**

In [ ]:
#Actualización Sep 2022:
#Se debe ejecutar esta línea primero para evitar un problema que comenzó a ocurrir hace unos meses.
#(Si al ejecutar sale un botón de "Reiniciar runtime", dar clic en él antes de continuar
!pip install pyyaml==5.4.1

In [ ]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 236, done.
remote: Total 236 (delta 0), reused 0 (delta 0), pack-reused 236
Receiving objects: 100% (236/236), 71.46 MiB | 28.52 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [ ]:
cd first-order-model

/content/first-order-model


In [ ]:
#Actualizacion 23-ago-2022: Agregar la librería de image-io :)
!pip install imageio-ffmpeg

**Montar el folder de Google Drive en este Colab**

In [ ]:
# Al ejecutar este bloque, aparecerá un URL. El URL se debe de abrir en el explorador
# y otorgar permisos. Ojo: Si te preocupa la seguridad, mejor hazlo en una cuenta de Google
# que te inventes solo para esto. Y subes las cosas en Google Drive de esa cuenta nueva.

# Cuando apruebes en Google Drive, te dara un codigo, y se debe pegar en el campo de texto
# que aparecera, y dar clic en Enter

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Agregar esta carpeta a tu Google Drive: https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing**

In [ ]:
# Aqui debes cargar el modelo que quieras.
# Por defecto tiene el de caras, pero puedes cambiarlo al de cuerpo, stickers, etc

# Una vez cargado se reutiliza para todas las veces que quieras ejecutar lo de abajo
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-model-checkpoints/vox-cpk.pth.tar')

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

**Cargar video, imagen destino, y animar :D**



In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

# Aqui se pone la imagen que quieres animar
source_image = imageio.imread('/content/gdrive/My Drive/deepfakes/putin.png')
# Aqui se pone el video fuente, que tiene el movimiento original
driving_video = imageio.mimread('/content/gdrive/My Drive/deepfakes/blabla.mp4')


# Por defecto la imagen y video se reducen a 256x256.
# Ojo! Revisa que ambos (video e imagen) sean cuadrados para que no batalles.

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]
    
#HTML(display(source_image, driving_video).to_html5_video())

from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])

# Mostrar abajo el video, es descargable
HTML(display(source_image, driving_video, predictions).to_html5_video())